In [1]:
import time
import os
from datetime import datetime

def capture_photo(cam_id, path, cam_name):
    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{cam_name}_{timestr}.jpg"
    filepath = os.path.join(path, filename)
    command = f"fswebcam -d v4l2:{cam_id} -r 1920x1080 --no-banner {filepath}"
    os.system(command)

# Define paths, camera IDs, and names
paths = [
    "/home/raspberry/bee/Media/Photos/Cam1",
    "/home/raspberry/bee/Media/Photos/Cam2",
    "/home/raspberry/bee/Media/Photos/Cam3"
]

cam_ids = [
    "/dev/video0",
    "/dev/video2",
    "/dev/video6"
]

cam_names = [
    "1Cam1",
    "2Cam2",
    "3Cam3"
]

def within_active_hours():
    current_time = datetime.now()
    start_time = current_time.replace(hour=7, minute=0, second=0, microsecond=0)
    end_time = current_time.replace(hour=18, minute=0, second=0, microsecond=0)
    return start_time <= current_time <= end_time

# Main loop for taking pictures
while True:
    if within_active_hours():
        for path, cam_id, cam_name in zip(paths, cam_ids, cam_names):
            capture_photo(cam_id, path, cam_name)
        # Sleep remains inside the if condition to ensure it runs every 10 seconds interval
        time.sleep(10)
    else:
        # Calculate the sleep duration until next run begins, i.e., between now and 7 AM the next day
        now = datetime.now()
        if now.hour >= 18:
            tomorrow = now.replace(day=now.day + 1, hour=7, minute=0, second=0, microsecond=0)
        else:
            tomorrow = now.replace(hour=7, minute=0, second=0, microsecond=0)

        sleep_duration = (tomorrow - now).total_seconds()
        print(f"Outside of active hours. Sleeping until 7 AM. Sleeping for {sleep_duration} seconds.")
        time.sleep(sleep_duration)


--- Opening v4l2:/dev/video0...
/dev/video0 opened.
No input was specified, using the first.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/raspberry/bee/Media/Photos/Cam1/1Cam1_2024-11-06_15-14-13.jpg'.
--- Opening v4l2:/dev/video2...
/dev/video2 opened.
No input was specified, using the first.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/raspberry/bee/Media/Photos/Cam2/2Cam2_2024-11-06_15-14-14.jpg'.
--- Opening v4l2:/dev/video6...
/dev/video6 opened.
No input was specified, using the first.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/raspberry/bee/Media/Photos/Cam3/3Cam3_2024-11-06_15-14-18.jpg'.
--- Opening v4l2:/dev/video0...
/dev/video0 opened.
No input was specified, using the first.
--- Capturing frame...
Captured

KeyboardInterrupt: 

In [ ]:
import time
import os
from datetime import datetime, timedelta
from picamera2 import Picamera2, Preview

def capture_photo_usb(cam_id, path, cam_name):
    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{cam_name}_{timestr}.jpg"
    filepath = os.path.join(path, filename)
    command = f"fswebcam -d v4l2:{cam_id} -r 1920x1080 --no-banner {filepath}"
    os.system(command)

def capture_photo_rpi(counter, base_path):
    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"RPICam_{timestr}_{counter}.jpg"
    filepath = os.path.join(base_path, filename)
    picam2.capture_file(filepath)

def within_active_hours():
    current_time = datetime.now()
    start_time = current_time.replace(hour=7, minute=0, second=0, microsecond=0)
    end_time = current_time.replace(hour=18, minute=0, second=0, microsecond=0)
    return start_time <= current_time <= end_time

def within_night_hours():
    current_time = datetime.now()
    start_night_time = current_time.replace(hour=18, minute=0, second=0, microsecond=0)
    if current_time.hour >= 18:
        end_night_time = (current_time + timedelta(days=1)).replace(hour=7, minute=0, second=0, microsecond=0)
    else:
        end_night_time = current_time.replace(hour=7, minute=0, second=0, microsecond=0)
    return start_night_time <= current_time < end_night_time

# Define paths, camera IDs, and names for USB cameras
usb_paths = [
    "/home/raspberry/bee/Media/Photos/Cam1",
    "/home/raspberry/bee/Media/Photos/Cam2",
    "/home/raspberry/bee/Media/Photos/Cam3"
]

usb_cam_ids = [
    "/dev/video0",
    "/dev/video2",
    "/dev/video6"
]

usb_cam_names = [
    "1Cam1",
    "2Cam2",
    "3Cam3"
]

# RPi Camera setup
picam2 = Picamera2()
camera_config = picam2.create_still_configuration(main={"size": (1640, 1232)}, lores={"size": (640, 480)}, display="lores")
picam2.configure(camera_config)
picam2.start_preview(Preview.QTGL)
picam2.start()
time.sleep(2)
rpi_counter = 1
rpi_path = "/home/raspberry/bee/Media/Photos/RPiCam"  # Set the path for saving RPi camera images

# Main loop for taking pictures
while True:
    if within_active_hours():
        for path, cam_id, cam_name in zip(usb_paths, usb_cam_ids, usb_cam_names):
            capture_photo_usb(cam_id, path, cam_name)
            time.sleep(2)
        time.sleep(8)  # Ensure total delay between captures from each camera is approximately 10 seconds
    elif within_night_hours():
        capture_photo_rpi(rpi_counter, rpi_path)
        rpi_counter += 1
        time.sleep(300)  # Sleep for 5 minutes (300 seconds)
    else:
        # Calculate sleep duration until next run begins
        now = datetime.now()
        if now.hour < 7:  # If the current time is before 7 AM
            next_start_time = now.replace(hour=7, minute=0, second=0, microsecond=0)
        else:  # If the current time is after 6 PM
            next_start_time = (now + timedelta(days=1)).replace(hour=7, minute=0, second=0, microsecond=0)

        sleep_duration = (next_start_time - now).total_seconds()
        print(f"Outside of active hours. Sleeping until 7 AM. Sleeping for {sleep_duration} seconds.")
        time.sleep(sleep_duration)


In [1]:
import os
import time
from picamera2 import Picamera2, Preview

# Directory where the test images will be saved
save_directory = "/home/raspberry/bee/Media/Photos/Test"  # Change this path as needed

# Ensure the save directory exists
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

def capture_photo_usb(cam_id, cam_name):
    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{cam_name}_{timestr}.jpg"
    filepath = os.path.join(save_directory, filename)
    command = f"fswebcam -d v4l2:{cam_id} -r 1920x1080 --no-banner {filepath}"
    os.system(command)
    print(f"Captured {filename} from {cam_name}")

def capture_photo_rpi(counter):
    picam2 = Picamera2()
    camera_config = picam2.create_still_configuration(main={"size": (1640, 1232)}, lores={"size": (640, 480)}, display="lores")
    picam2.configure(camera_config)
    picam2.start_preview(Preview.QTGL)
    picam2.start()
    time.sleep(2)

    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"RPICam_{timestr}_{counter}.jpg"
    filepath = os.path.join(save_directory, filename)
    picam2.capture_file(filepath)
    print(f"Captured {filename} from Raspberry Pi Camera")

# Define camera IDs and names for USB cameras
usb_cam_ids = [
    "/dev/video0",
    "/dev/video2",
    "/dev/video6"
]

usb_cam_names = [
    "1Cam1",
    "2Cam2",
    "3Cam3"
]

# Identify a counter for the RPi camera image
rpi_counter = 1

# Capture images from all cameras
for cam_id, cam_name in zip(usb_cam_ids, usb_cam_names):
    capture_photo_usb(cam_id, cam_name)
    time.sleep(2)  # Give some time between captures

capture_photo_rpi(rpi_counter)


--- Opening v4l2:/dev/video0...
/dev/video0 opened.
No input was specified, using the first.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/raspberry/bee/Media/Photos/Test/1Cam1_2024-11-06_15-35-44.jpg'.


Captured 1Cam1_2024-11-06_15-35-44.jpg from 1Cam1


--- Opening v4l2:/dev/video2...
/dev/video2 opened.
No input was specified, using the first.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/raspberry/bee/Media/Photos/Test/2Cam2_2024-11-06_15-35-47.jpg'.


Captured 2Cam2_2024-11-06_15-35-47.jpg from 2Cam2


--- Opening v4l2:/dev/video6...
/dev/video6 opened.
No input was specified, using the first.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/raspberry/bee/Media/Photos/Test/3Cam3_2024-11-06_15-35-50.jpg'.


Captured 3Cam3_2024-11-06_15-35-50.jpg from 3Cam3


[5:59:13.193017859] [421246]  INFO Camera camera_manager.cpp:325 libcamera v0.3.2+27-7330f29b
[5:59:13.303757833] [426296]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[5:59:13.305733382] [426296]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[5:59:13.306366682] [426296]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media3 and ISP device /dev/media0
[5:59:13.306411959] [426296]  INFO RPI pipeline_base.cpp:1126 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[5:59:13.563387121] [421246]  INFO Camera camera.cpp:1197 configuring streams: (0) 1640x1232-BGR888 (1) 640x480-YUV420 (2) 1640x1232-SBGGR10_CSI2P
[5:59:13.564786146] [426296]  INFO RPI vc4.cpp:622 Sensor: /base/soc/i2c0mux/i2c@1/imx219@10 - Selected sensor format: 1640x1232-SBGGR10_1X10 - Selected unicam format: 1640x1232-pBAA


Captured RPICam_2024-11-06_15-36-01_1.jpg from Raspberry Pi Camera
